In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Neste exemplos usaremos a API sequencial, mas o método `save()` funciona também para a API funcional.

In [4]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(30, activation="relu", input_shape=[8]),
        keras.layers.Dense(30, activation="relu"),
        keras.layers.Dense(1)
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Treinando o modelo.

In [5]:
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    validation_data=(X_valid, y_valid)
                   )

Epoch 1/10
363/363 [==============================] - 3s 4ms/step - loss: 1.8866 - val_loss: 0.7126
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6577 - val_loss: 0.6880
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5934 - val_loss: 0.5803
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5557 - val_loss: 0.5166
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5272 - val_loss: 0.4895
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5033 - val_loss: 0.4951
Epoch 7/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4854 - val_loss: 0.4861
Epoch 8/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4709 - val_loss: 0.4554
Epoch 9/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4578 - val_loss: 0.4413
Epoch 10/10
363/363 [==============================] - 2s 4ms/step - loss: 0.4474 - val_loss: 0.4379

### Salvando o modelo.

In [6]:
model.save("./my_keras_model.h5")

### Carregando o modelo salvo.

In [7]:
loaded_model = keras.models.load_model("./my_keras_model.h5")

### Realizando predições com o modelo salvo.

In [8]:
X_new = X_test[:3]

y_pred = loaded_model.predict(X_new)

1/1 [==============================] - 0s 334ms/step


In [9]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

Actual: 0.477 - Predicted: 0.540
Actual: 0.458 - Predicted: 1.651
Actual: 5.000 - Predicted: 3.010


### Salvando os pesos de modelos criados a partir de subclasses da classe `Model`.

+ O método `save()` e a função `load_model` funcionam ao usar as APIs sequencial e funcional, **mas, infelizmente, não ao usar subclasses da classe `Model`.** 

+ Porém, podemos usar os métodos `save_weights()` e `load_weights()` para pelo menos salvar e restaurar os parâmetros do modelo. 

+ Entretanto, precisaremos salvar e restaurar todo o resto nós mesmos.

In [10]:
class Regressor(keras.models.Model):
    
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # lida com parâmetros padrão como, por exemplo, name.
        self.hidden = keras.layers.Dense(units, activation=activation, name="input")
        self.out = keras.layers.Dense(1)       
        
    def call(self, inputs):
        hidden = self.hidden(inputs)
        out = self.out(hidden)
        return out

In [11]:
model = Regressor()

In [12]:
model.compile(
    loss="mse", 
    optimizer=keras.optimizers.SGD(learning_rate=1e-3)
)

In [13]:
# Treinando o modelo.
history = model.fit(X_train, 
                    y_train, 
                    epochs=20,
                    validation_data=(X_valid, y_valid)
                   )

Epoch 1/20
363/363 [==============================] - 5s 9ms/step - loss: 1.7566 - val_loss: 0.8978
Epoch 2/20
363/363 [==============================] - 2s 5ms/step - loss: 0.8452 - val_loss: 1.0379
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7734 - val_loss: 0.8684
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.7197 - val_loss: 0.7182
Epoch 5/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6758 - val_loss: 0.6508
Epoch 6/20
363/363 [==============================] - 2s 6ms/step - loss: 0.6374 - val_loss: 0.6208
Epoch 7/20
363/363 [==============================] - 2s 5ms/step - loss: 0.6046 - val_loss: 0.6002
Epoch 8/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5756 - val_loss: 0.5443
Epoch 9/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5497 - val_loss: 0.5202
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5274 - val_loss: 0.5022

In [14]:
model.save_weights("./my_keras_weights.ckpt")

### Importante

+ Para carregar os pesos, **precisamos antes ter um modelo instanciado e compilado**.

In [15]:
loaded_model = Regressor()

loaded_model.compile(
    loss="mse", 
    optimizer=keras.optimizers.SGD(learning_rate=1e-3)
)

loaded_model.load_weights("./my_keras_weights.ckpt")

Depois dos pesos terem sido carregados, podemos fazer predições normalmente.

In [16]:
y_pred = loaded_model.predict(X_new)

1/1 [==============================] - 0s 173ms/step


In [18]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i]))

Actual: 0.477 - Predicted: 0.481
Actual: 0.458 - Predicted: 1.703
Actual: 5.000 - Predicted: 3.254
